# 第7章

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

### CSVデータを読みこむ

In [2]:
# データの読みこみ
dau = pd.read_csv("../Data/section7-dau.csv")
dau.head()

,region_month,region_day,app_name,user_id,device
0,2013-01,2013-01-01,game-02,10061580,FP
1,2013-01,2013-01-01,game-02,10154440,FP
2,2013-01,2013-01-01,game-02,10164762,SP
3,2013-01,2013-01-01,game-02,10165615,FP
4,2013-01,2013-01-01,game-02,10321356,FP


### ユーザ別に、移行した人かどうかのデータの整理

In [3]:
# MAU
mau = dau[["region_month", "device", "user_id"]].drop_duplicates()
mau.head()

,region_month,device,user_id
0,2013-01,FP,10061580
1,2013-01,FP,10154440
2,2013-01,SP,10164762
3,2013-01,FP,10165615
4,2013-01,FP,10321356


In [4]:
#FP MAU
fp_mau = mau.query("device == 'FP'")
fp_mau.head()

,region_month,device,user_id
0,2013-01,FP,10061580
1,2013-01,FP,10154440
3,2013-01,FP,10165615
4,2013-01,FP,10321356
6,2013-01,FP,10447112


In [5]:
#SP MAU
sp_mau = mau.query("device == 'SP'")
sp_mau.head()

,region_month,device,user_id
2,2013-01,SP,10164762
5,2013-01,SP,10406653
8,2013-01,SP,10494712
11,2013-01,SP,10575741
12,2013-01,SP,10588037


In [6]:
# 1月と2月で分ける
fp_mau1 = fp_mau.query("region_month == '2013-01'")
fp_mau2 = fp_mau.query("region_month == '2013-02'")

sp_mau1 = sp_mau.query("region_month == '2013-01'")
sp_mau2 = sp_mau.query("region_month == '2013-02'")

In [7]:
# 1月携帯電話からの利用で2月にアクセスがあるかどうか
mau["is_access"] = 1
fp_mau1 = pd.merge(fp_mau1, mau.query("region_month == '2013-02'")[["user_id", "is_access"]], on = "user_id", how = "left")
fp_mau1 = fp_mau1.fillna({"is_access": 0})
fp_mau1.head()

,region_month,device,user_id,is_access
0,2013-01,FP,10061580,1.0
1,2013-01,FP,10154440,0.0
2,2013-01,FP,10165615,1.0
3,2013-01,FP,10321356,1.0
4,2013-01,FP,10447112,1.0


In [8]:
# 1月は携帯電話からの利用で2月にも携帯電話からの利用があったかどうか
fp_mau2["is_fp"] = 1
fp_mau1 = pd.merge(fp_mau1, fp_mau2[["user_id", "is_fp"]], on = "user_id", how = "left")
fp_mau1 = fp_mau1.fillna({"is_fp": 0})
fp_mau1.head()

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,region_month,device,user_id,is_access,is_fp
0,2013-01,FP,10061580,1.0,1.0
1,2013-01,FP,10154440,0.0,0.0
2,2013-01,FP,10165615,1.0,1.0
3,2013-01,FP,10321356,1.0,1.0
4,2013-01,FP,10447112,1.0,1.0


In [9]:
# 1月は携帯電話からの利用で2月にスマートフォンから利用があったかどうか
sp_mau2["is_sp"] = 1
fp_mau1 = pd.merge(fp_mau1, sp_mau2[["user_id", "is_sp"]], on = "user_id", how = "left")
fp_mau1 = fp_mau1.fillna({"is_sp": 0})
fp_mau1.head()

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,region_month,device,user_id,is_access,is_fp,is_sp
0,2013-01,FP,10061580,1.0,1.0,0.0
1,2013-01,FP,10154440,0.0,0.0,0.0
2,2013-01,FP,10165615,1.0,1.0,0.0
3,2013-01,FP,10321356,1.0,1.0,0.0
4,2013-01,FP,10447112,1.0,1.0,0.0


In [10]:
# 1月携帯電話利用で2月は利用なし、あるいはスマートフォン利用があったユーザ
fp_mau1 = fp_mau1.query("is_access == 0 | is_sp == 1")
fp_mau1.head()

,region_month,device,user_id,is_access,is_fp,is_sp
1,2013-01,FP,10154440,0.0,0.0,0.0
7,2013-01,FP,10528830,0.0,0.0,0.0
20,2013-01,FP,1163733,1.0,0.0,1.0
21,2013-01,FP,11727630,0.0,0.0,0.0
43,2013-01,FP,13401362,1.0,0.0,1.0


### 日ごとにゲームにアクセスしたかどうかのデータを整える

In [11]:
fp_dau1 = dau.query("device == 'FP' & region_month == '2013-01'")
fp_dau1["is_access"] = 1

fp_dau1.head()

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,region_month,region_day,app_name,user_id,device,is_access
0,2013-01,2013-01-01,game-02,10061580,FP,1
1,2013-01,2013-01-01,game-02,10154440,FP,1
3,2013-01,2013-01-01,game-02,10165615,FP,1
4,2013-01,2013-01-01,game-02,10321356,FP,1
6,2013-01,2013-01-01,game-02,10447112,FP,1


In [12]:
fp_dau1_cast = fp_dau1.groupby(["user_id", "region_day"])["is_access"].max().unstack().reset_index().fillna(0)
fp_dau1_cast.columns = ["user_id"] + ["X%dday" % d for d in range(1, 32)]
fp_dau1_cast.head()

,user_id,X1day,X2day,X3day,X4day,X5day,X6day,X7day,X8day,X9day,...,X22day,X23day,X24day,X25day,X26day,X27day,X28day,X29day,X30day,X31day
0,397286,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,471341,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,503874,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,512250,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,513811,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


In [13]:
# 2月利用でかつスマホからの利用者をくっつける
fp_dau1_cast = pd.merge(fp_dau1_cast, fp_mau1[["user_id", "is_sp"]], on = "user_id")
fp_dau1_cast.head()

,user_id,X1day,X2day,X3day,X4day,X5day,X6day,X7day,X8day,X9day,...,X23day,X24day,X25day,X26day,X27day,X28day,X29day,X30day,X31day,is_sp
0,471341,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,503874,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1073544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1073864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1163733,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0


In [14]:
fp_dau1_cast[["user_id", "is_sp"]].groupby("is_sp").count()

,user_id
is_sp,
0.0,190
1.0,62


### ロジスティック回帰分析によるモデル作成

In [15]:
model = smf.glm(formula = 'is_sp ~ X1day + X2day + X3day + X4day + X5day + X6day + X7day + X8day + X9day + X10day + X11day + X12day + X13day + X14day + X15day + X16day + X17day + X18day + X19day + X20day + X21day + X22day + X23day + X24day + X25day + X26day + X27day + X28day + X29day + X30day + X31day', data = fp_dau1_cast, family = sm.families.Binomial())
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  is_sp   No. Observations:                  252
Model:                            GLM   Df Residuals:                      220
Model Family:                Binomial   Df Model:                           31
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -56.166
Date:                Fri, 18 May 2018   Deviance:                       112.33
Time:                        10:45:29   Pearson chi2:                     185.
No. Iterations:                     7                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.2551      0.592     -7.192      0.000      -5.415      -3.096
X1day          1.9873      0.804      2.471      0.013       0.411       3.564
X2day          0.3245      1.070      0.303      0.762      -1.774       2.423
X3day          0.3871      0.940      0.412      0.681      -1.455       2.229
X4day          2.0520      0.836      2.455      0.014       0.414       3.690
X5day         -1.7603      1.202     -1.464      0.143      -4.116       0.596
X6day         -0.6650      1.172     -0.567      0.570      -2.962       1.632
X7day          2.0119      1.120      1.796      0.072      -0.184       4.207
X8day          1.2009      1.352      0.888      0.375      -1.450       3.852
X9day         -0.4430      1.187     -0.373      0.709      -2.769       1.883
X10day        -3.2135      1.555     -2.067      0.039      -6.261      -0.166
X11day         1.4348      1.224      1.172      0.241      -0.964       3.834
X12day        -0.6478      1.530     -0.423      0.672      -3.647       2.351
X13day         1.7741      1.144      1.551      0.121      -0.467       4.015
X14day        -1.1306      1.248     -0.906      0.365      -3.576       1.315
X15day        -1.8019      1.303     -1.382      0.167      -4.357       0.753
X16day         0.4899      1.167      0.420      0.675      -1.798       2.778
X17day        -0.4389      1.222     -0.359      0.719      -2.834       1.956
X18day         1.4439      1.145      1.261      0.207      -0.801       3.688
X19day        -0.5965      1.196     -0.499      0.618      -2.941       1.748
X20day        -0.3477      1.400     -0.248      0.804      -3.092       2.397
X21day         0.9714      1.254      0.775      0.439      -1.487       3.430
X22day         1.1868      1.123      1.057      0.291      -1.014       3.388
X23day        -1.5476      1.233     -1.256      0.209      -3.964       0.868
X24day         1.8509      1.102      1.680      0.093      -0.309       4.011
X25day         0.1355      1.532      0.088      0.930      -2.868       3.139
X26day         0.3079      1.429      0.215      0.829      -2.494       3.109
X27day         0.3539      1.295      0.273      0.785      -2.184       2.892
X28day        -0.4434      1.854     -0.239      0.811      -4.076       3.190
X29day         1.6132      1.055      1.529      0.126      -0.455       3.682
X30day         2.0081      1.211      1.658      0.097      -0.366       4.382
X31day         1.5179      1.182      1.284      0.199      -0.799       3.835
==============================================================================
"""

### 作成されたモデルを利用した予測

In [16]:
# SP移行確率
fp_dau1_cast["prob"] = result.predict(fp_dau1_cast)
fp_dau1_cast["pred"] = np.where(fp_dau1_cast["prob"] > 0.5, 1, 0)
fp_dau1_cast.head()

,user_id,X1day,X2day,X3day,X4day,X5day,X6day,X7day,X8day,X9day,...,X25day,X26day,X27day,X28day,X29day,X30day,X31day,is_sp,prob,pred
0,471341,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.621464,1
1,503874,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.093830,0
2,1073544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000884,0
3,1073864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018876,0
4,1163733,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.867287,1


In [17]:
# 予測と実際
fp_dau1_cast.pivot_table(index = "is_sp", columns = "pred", values = "user_id", aggfunc = len)

pred,0,1
is_sp,,
0.0,180,10
1.0,14,48


###  予測結果からユーザ群を推測する

In [18]:
fp_dau1_cast1 = fp_dau1_cast.query("is_sp == 1 & pred == 1")
fp_dau1_cast1.sort_values(by = "prob", ascending = False).head()

,user_id,X1day,X2day,X3day,X4day,X5day,X6day,X7day,X8day,X9day,...,X25day,X26day,X27day,X28day,X29day,X30day,X31day,is_sp,prob,pred
136,24791702,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.999561,1
137,24791702,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.999561,1
43,9567562,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.998269,1
44,9567562,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.998269,1
115,21551429,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.996861,1


In [19]:
fp_dau1_cast2 = fp_dau1_cast.query("is_sp == 0 & pred == 1")
fp_dau1_cast2.sort_values(by = "prob", ascending = False).head()

,user_id,X1day,X2day,X3day,X4day,X5day,X6day,X7day,X8day,X9day,...,X25day,X26day,X27day,X28day,X29day,X30day,X31day,is_sp,prob,pred
194,41590801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.737742,1
209,46285446,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.636544,1
108,19432099,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.630625,1
14,3955950,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.621464,1
158,28391896,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.546836,1


In [20]:
fp_dau1_cast3 = fp_dau1_cast.query("is_sp == 0 & pred == 0")
fp_dau1_cast3.sort_values(by = "prob").head()

,user_id,X1day,X2day,X3day,X4day,X5day,X6day,X7day,X8day,X9day,...,X25day,X26day,X27day,X28day,X29day,X30day,X31day,is_sp,prob,pred
149,27249550,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000206,0
101,18408297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000534,0
242,60725457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000570,0
10,2541741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000570,0
172,33766090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000755,0
